# [Introduction To LangChain](https://docs.langchain.com/docs/)

<br>

- LangChain is a framework for developing applications powered by language models.

## Topics Covered

1. Making calls directly to OpenAI LLM.
2. Using LangChain to create prompt templates and schemas
3. Using LangChain to create output parsers.
4. Using LangChain to create memory.
5. Using LangChain to create chains.

## Installation

```sh
pip install langchain

# OR
pip install 'langchain[all]'

# Other dependencies
pip install python-dotenv
pip install openai
```

In [ ]:
# Built-in library
import itertools
import re
import json
from typing import Any, Dict, List, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
from pprint import pprint
import pandas as pd

# Visualization
import matplotlib.pyplot as plt


# pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black
# auto reload imports
%load_ext autoreload
%autoreload 2

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate

from langchain.output_parsers import ResponseSchema, StructuredOutputParser

from langchain.chains import ConversationChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains import SequentialChain, SimpleSequentialChain
from langchain.chains import LLMChain


from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryBufferMemory,
    ConversationTokenBufferMemory,
    ConversationBufferWindowMemory,
)

### [Language models](https://python.langchain.com/docs/modules/model_io/models/)

```text
LangChain provides interfaces and integrations for two types of models:

1. LLMs: Models that take a text string as input and return a text string

2. Chat models: Models that are backed by a language model but take a list of Chat Messages as input and return a Chat Message.
```


#### [LLMs](https://python.langchain.com/docs/modules/model_io/models/llms/)

In [ ]:
def get_completion(prompt: str, model: str = "gpt-3.5-turbo"):
    """This is used to make a direct API calls to OpenAI."""
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [ ]:
get_completion("What is 1+1?")

'1+1 equals 2.'